In [14]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.itemknn.itemknn import ItemKNN
from src.models.ease.ease import EASE
import pandas as pd
from src.evaluation import compute_mrr
from src.utils.sparse_matrix import interactions_to_sparse_matrix

In [15]:
dataset = Dataset()

In [16]:
split_dict = dataset.get_split()
train, train_label = split_dict[TRAIN]
val, val_label = split_dict[VAL]
test, test_label = split_dict[TEST]

In [38]:
mask = (train.groupby(SESS_ID).count()[ITEM_ID]>=3).reset_index()

In [44]:
filtered_sess = mask[mask[ITEM_ID]][SESS_ID].values

In [45]:
filtered_sess

array([     18,      19,      24, ..., 4439990, 4439999, 4440001])

In [17]:
from datetime import datetime, timedelta
max_date = train[DATE].max()
train_limit_date = max_date - timedelta(days=120)

In [18]:
train_limit_date

Timestamp('2020-12-31 23:41:03.992000')

In [19]:
filtered_train = train[train[DATE]>train_limit_date].copy()
id_filtered_train = filtered_train[SESS_ID].unique()
filtered_train_label = train_label[train_label[SESS_ID].isin(id_filtered_train)].copy()


In [46]:
filtered_length_train = train[train[SESS_ID].isin(filtered_sess)]
filtered_length_train_label = train_label[train_label[SESS_ID].isin(filtered_sess)]

In [47]:
filtered_length_train

,session_id,item_id,date,sample_weight
3,18,15459,2020-08-26 19:18:30.833,1
4,18,2085,2020-08-26 19:16:31.211,1
5,18,3377,2020-08-26 19:15:47.232,1
6,19,21726,2020-11-02 16:31:18.543,1
7,19,5336,2020-11-02 16:34:33.794,1
...,...,...,...,...
4273672,4440001,17238,2020-10-30 23:37:20.658,1
4273673,4440001,11916,2020-10-30 23:31:56.607,1
4273674,4440001,12039,2020-10-30 23:36:17.934,1
4273675,4440001,23434,2020-10-30 23:39:55.186,1


In [20]:
full_data = dataset.get_train_sessions()
# we can not use that for the final submission
lead_data = dataset.get_test_leaderboard_sessions()
final_data = dataset.get_test_final_sessions()

In [48]:
train["sample_weight"] = 1
train_label["sample_weight"] = 5

filtered_train["sample_weight"] = 1
filtered_train_label["sample_weight"] = 5

filtered_length_train["sample_weight"] = 1
filtered_length_train_label["sample_weight"] = 5

val["sample_weight"] = 1
val_label["sample_weight"] = 5

test["sample_weight"] = 1
test_label["sample_weight"] = 5

lead_data["sample_weight"] = 1
final_data["sample_weight"] = 1

/tmp/ipykernel_2646431/2297539540.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_length_train["sample_weight"] = 1
/tmp/ipykernel_2646431/2297539540.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_length_train_label["sample_weight"] = 5


In [49]:
train_pur = pd.concat([filtered_length_train, filtered_length_train_label], axis=0)

In [50]:
model = EASE(dataset, time_weight=50, l2=1e-1)

In [51]:
model.compute_similarity_matrix(train_pur)

In [52]:
recs = model.recommend(
    interactions=val,
    remove_seen=True,
    cutoff=100,
    leaderboard=True
)

Using Time Weight on Interaction matrix
Considering white list items...


In [53]:
compute_mrr(recs, val_label)

MRR: 0.15455163411537248


0.15455163411537248

# Create Submission

In [28]:
concat_full_data = pd.concat([filtered_train, filtered_train_label, val, val_label, test, test_label], axis=0)
#concat_full_data = pd.concat([train, train_label, val, val_label, test, test_label], axis=0)

In [29]:
model = EASE(dataset, time_weight=50, l2=1e-1)

In [30]:
model.compute_similarity_matrix(concat_full_data)

In [31]:
recs = model.recommend(
    interactions=lead_data,
    remove_seen=True,
    cutoff=100,
    leaderboard=True
)

Using Time Weight on Interaction matrix
Considering white list items...


In [32]:
dataset.create_submission(recs, sub_name="Ease_3m")

Submission with name: Ease_3m created succesfully!
